# TF-IDF 활용 핵심 키워드 추출

## 실습1. sklearn활용

In [1]:
import pandas as pd

In [2]:
import requests 
from bs4 import BeautifulSoup
import re

def get_news_by_url(url):
  headers={"user-agent":"Mozilla/5.0"}
  res = requests.get(url, headers=headers)
  soup = BeautifulSoup(res.content, "html.parser")
  content = soup.select_one("#articleBodyContents").get_text().replace("\n", "")
  content = content.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
  
  start_pos = re.search(r"\w+@\w+\.\w+(.\w+)?", content).start()
  content = content[:start_pos-1]
  return content

docs = []
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=001&aid=0011614790') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=014&aid=0004424362') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=119&aid=0002402191') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=030&aid=0002882728') )
len(docs)

5

### 전처리

In [4]:
from konlpy.tag import Mecab
mecab = Mecab()

preprocessed_docs = []


for doc in docs:
  token_list = []
  for token in mecab.pos(doc):
    if token[1] in ['NNG', 'VV']:
      token_list.append(token[0])
  preprocessed_docs.append(" ".join(token_list))


preprocessed_docs[0][:100]

'과기 정통부 장관 참석 기념행사 투입 여종 데이터 구축 민간 외부 연계 체계 개방 강화 기자 국가 차원 빅 데이터 활용 시대 산업 창출 기존 산업 변화 이르 혁신 장 센터 문 분야'

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(max_df=0.85, max_features=10000)
word_count = count_vect.fit_transform(preprocessed_docs)
print((count_vect.get_feature_names()[:10]))


['가공', '가능', '가입자', '가족', '가중치', '가치', '각종', '감소', '감염', '강국']


### TF-IDF 계산

In [8]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer.fit(word_count)


TfidfTransformer()

### 핵심 키워드 추출

In [9]:
def sort_keywords(keywords):
    return sorted(zip(keywords.col, keywords.data), key=lambda x: (x[1], x[0]), reverse=True)
 
def extract_keywords(feature_names, sorted_keywords, n=5):
    pass

In [27]:
doc = preprocessed_docs[0] # 핵심키워드 추출할 문서 조회

feature_names = count_vect.get_feature_names()
tfidf_vect = tfidf_transformer.transform(count_vect.transform([doc]))
sort_keywords(tfidf_vect.tocoo())[:10]


# 사용자가 지정한 갯수만큼 키워드 추출
keywords = extract_keywords(feature_names, sorted_keywords, 5)
 
print("\n===== 원문 =====")
print(docs[0][:100])
print("\n=== 핵심키워드 ===")
for k in keywords:
    print(k)

TypeError: 'coo_matrix' object is not subscriptable

In [19]:
tfidf_vect

<1x393 sparse matrix of type '<class 'numpy.float64'>'
	with 201 stored elements in COOrdinate format>

In [12]:
tfidf_vect.tocoo().col

array([391, 387, 385, 382, 380, 379, 378, 377, 375, 374, 373, 372, 371,
       370, 369, 365, 362, 361, 360, 359, 358, 355, 354, 352, 349, 347,
       346, 345, 338, 337, 336, 335, 333, 332, 330, 328, 327, 326, 325,
       323, 321, 320, 319, 317, 316, 314, 308, 307, 306, 301, 299, 294,
       289, 288, 286, 285, 284, 283, 282, 281, 280, 276, 274, 270, 268,
       265, 263, 259, 256, 251, 247, 246, 245, 241, 240, 239, 237, 234,
       233, 231, 229, 228, 227, 226, 223, 222, 221, 219, 216, 214, 213,
       212, 211, 210, 209, 204, 203, 198, 195, 194, 192, 189, 185, 184,
       182, 181, 180, 179, 178, 177, 175, 172, 171, 166, 163, 162, 161,
       160, 152, 151, 150, 147, 146, 144, 142, 139, 135, 134, 132, 129,
       128, 126, 122, 121, 118, 115, 113, 110, 109, 108, 107, 105, 103,
       102, 101, 100,  98,  95,  91,  88,  87,  86,  83,  81,  80,  77,
        76,  75,  73,  71,  68,  67,  64,  62,  61,  59,  58,  55,  54,
        52,  51,  49,  48,  47,  46,  45,  43,  41,  38,  37,  3

In [20]:
tfidf_vect.tocoo().data

array([0.03703823, 0.18780025, 0.07407646, 0.07407646, 0.03703823,
       0.03703823, 0.03703823, 0.02988221, 0.03703823, 0.14941106,
       0.03703823, 0.03703823, 0.02988221, 0.03703823, 0.07407646,
       0.03703823, 0.05976443, 0.03703823, 0.25926763, 0.03703823,
       0.02988221, 0.03703823, 0.02988221, 0.02988221, 0.03703823,
       0.1111147 , 0.03703823, 0.1111147 , 0.03703823, 0.02988221,
       0.03703823, 0.07407646, 0.03703823, 0.04173339, 0.02988221,
       0.07407646, 0.03703823, 0.03703823, 0.1111147 , 0.03703823,
       0.1111147 , 0.03703823, 0.03703823, 0.02988221, 0.03703823,
       0.03703823, 0.02988221, 0.02988221, 0.03703823, 0.03703823,
       0.1111147 , 0.07407646, 0.02988221, 0.03703823, 0.12402468,
       0.07407646, 0.18519116, 0.02988221, 0.04173339, 0.02988221,
       0.08964664, 0.07407646, 0.03703823, 0.03703823, 0.03703823,
       0.02988221, 0.1111147 , 0.03703823, 0.03703823, 0.02988221,
       0.03703823, 0.03703823, 0.03703823, 0.03703823, 0.02480

In [15]:
class A():
    def __init__(self):
        print("A")
    def __str__(self):
        return "I am A"

In [17]:
a = A()
print(A)

A
<class '__main__.A'>


## gensim 활용

### 전처리

In [21]:
from konlpy.tag import Mecab
mecab = Mecab()

preprocessed_docs = []
for doc in docs :
  # 명사와 동사만으로 문서 전처리
  preprocessed_docs.append(' '.join([token[0] for token in mecab.pos(doc) if token[1][0] in ['N', 'V']]))
preprocessed_docs[0][:100]

'과기 정통부 일 유영민 장관 등 참석 기념행사 년 억 원 투입 여종 데이터 구축 민간 클라우드 통한 외부 연계 체계 개방 강화 데일리 이재운 기자 국가 차원 빅 데이터 활용 시대 '

### TF-IDF 계산

In [24]:
from gensim.models import TfidfModel
from gensim.corpora import Dictionary

document_ls = [doc.split() for doc in preprocessed_docs]
dct = Dictionary(document_ls) # 인덱스(key) - 단어(valuue) 인 딕셔너리 생성
corpus = [dct.doc2bow(doc) for doc in document_ls] # 각 문서에 포함된 단어를 인덱스로 변환하여 corpus 생성
tfidf = TfidfModel(corpus) # TF-IDF 산출

/Users/hosung/opt/anaconda3/envs/nlp/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [25]:
def sort_keywords(tfidf):
    return sorted(tfidf, key=lambda x: (x[1], x[0]), reverse=True)

def extract_keywords(feature_names, sorted_keywords, n=5):
    return [(feature_names[idx], score) for idx, score in sorted_keywords[:n]]

In [26]:
doc = corpus[0]

sorted_keywords = sort_keywords(tfidf[doc]) # TF-IDF를 기준으로 역순 정렬

# 사용자가 지정한 갯수만큼 키워드 추출
keywords = extract_keywords(dct, sorted_keywords, 5)
print(keywords)
print("\n==== 원문 =====")

print("\n=== 핵심키워드 ===")
for k in keywords:
    print(k)


=== 핵심키워드 ===
('플랫', 0.2495222182663338)
('폼', 0.2495222182663338)
('계획', 0.21387618708542896)
('정통부', 0.17823015590452412)
('위한', 0.17823015590452412)


In [28]:
tokens = ['딸기', '바나나', '사과', '딸기', '파인애플']
nodes = ['바나나', '사과', '파인애플', '딸기']
vocab = nodes

vocab2idx = {vocab[i]:i for i in range(0, len(vocab))} #vocab을 인덱스로 변환
idx2vocab = {i:vocab[i] for i in range(0, len(vocab))} #인덱스를 vocab으로 변환
vocab2idx

{'바나나': 0, '사과': 1, '파인애플': 2, '딸기': 3}

3) 그래프 생성 (weighted edge 계산)

- TextRank는 그래프 기반 모델
- 각 단어(토큰)은 그래프의 노드(vertex)
- weighted_edge 행렬은 노드간 가중치 정보를 담고 있음
- weighted_edge[i][j] 는 i번째 단어와 j번째 단어의 가중치를 의미
- weighted_edge[i][j] 가 0인 경우는 노드간 연결이 없음을 의미
- 모든 노드는 1로 초기화

In [31]:
import numpy as np
import math
vocab_len = len(vocab)

# 토큰별로 그래프 edge를 Matrix 형태로 생성
weighted_edge = np.zeros((vocab_len,vocab_len),dtype=np.float32)

# 각 토큰 노드별로 스코어 1로 초기화
score = np.ones((vocab_len),dtype=np.float32)

# coocurrence를 판단하기 위한 window 사이즈 설정
window_size = 2
covered_cooccurence = []

for window_start in range(0, (len(tokens) - window_size + 1)):
    window = tokens[window_start : window_start + window_size]
    print(window)
    
    for i in range(window_size):
        for j in range(i + 1, window_size):
            if(window[i] in vocab and window[j] in vocab):
                index_i = i + window_start
                index_j = j + window_start
                
                if [index_i, index_j] not in covered_cooccurence:
                    weighted_edge[vocab2idx[window[i]]][vocab2idx[window[j]]] = 1
                    weighted_edge[vocab2idx[window[j]]][vocab2idx[window[i]]] = 1
                    covered_cooccurence.append((index_i, index_j))
                    
for i in range(vocab_len):
    row_sum = weighted_edge[i].sum()
    print(f"{i} : {row_sum}")
    weighted_edge[i] = weighted_edge[i]/row_sum
    
print(weighted_edge)

['딸기', '바나나']
['바나나', '사과']
['사과', '딸기']
['딸기', '파인애플']


array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)

### 핵심 단어 추출

In [2]:
print(np.argsort(score))
sorted_index = np.flip(np.argsort(score), 0)

n = 4
 print("\n ===핵심 키워드 ===")
    for i in range(0, n):
        print(str(idx2vocab[sorted_index[i]])+" : " + str(score[sorted_index[i]]))

IndentationError: unexpected indent (<ipython-input-2-af9387c06e74>, line 5)